In [79]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

In [80]:
# Import train and test set

df_train = pd.read_csv('../raw_data/aug_train.csv')
df_test = pd.read_csv('../raw_data/aug_test.csv')
print(df_train.shape, df_test.shape)

(19158, 14) (2129, 13)


# I. Basic Info about Dataset

In [81]:
df_train.columns

Index(['enrollee_id', 'city', 'city_development_index', 'gender',
       'relevent_experience', 'enrolled_university', 'education_level',
       'major_discipline', 'experience', 'company_size', 'company_type',
       'last_new_job', 'training_hours', 'target'],
      dtype='object')

In [82]:
# Create list of ids that belong to each df

test_id = df_test.enrollee_id.values.tolist()
train_id = df_test.enrollee_id.values.tolist()

In [83]:
# Merge train and test for cleaning
df = df_train.merge(df_test, how='outer', on=['enrollee_id', 'city', 'city_development_index', 'gender', 
                                              'relevent_experience', 'enrolled_university', 'education_level',
                                              'major_discipline', 'experience', 'company_size', 'company_type',
                                              'last_new_job', 'training_hours'])
df.shape

(21287, 14)

In [84]:
df.head()

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,8949,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevent experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevent experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevent experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevent experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0


In [85]:
# Number of NAs in each column

perc_na = round(df.isna().sum() * 100 / len(df), 2)

pd.DataFrame({'Total NA':df.isna().sum(), 'DType':df.dtypes, '% NA':perc_na})

,Total NA,DType,% NA
enrollee_id,0,int64,0.00
city,0,object,0.00
city_development_index,0,float64,0.00
gender,5016,object,23.56
relevent_experience,0,object,0.00
enrolled_university,417,object,1.96
education_level,512,object,2.41
major_discipline,3125,object,14.68
experience,70,object,0.33
company_size,6560,object,30.82


Gender, company size, and company type all have high % of null values. None of the numeric columns have missing values (with the exception of the test group in target).

# II. Explore each column

In [86]:
# To keep track of which columns need to be encoded
cat_cols = []
ord_cols = []

### 1) Enrollee_id

In [87]:
# Convert enrollee_id to string
df['enrollee_id'] = df['enrollee_id'].astype(str)

### 2) City

In [88]:
df.groupby(by='city').mean()['city_development_index']

city
city_1      0.847
city_10     0.895
city_100    0.887
city_101    0.558
city_102    0.804
            ...  
city_93     0.865
city_94     0.698
city_97     0.925
city_98     0.949
city_99     0.915
Name: city_development_index, Length: 123, dtype: float64

In [89]:
pd.DataFrame(
    {'Total':df['city'].value_counts(),
     '% of Total':round(df['city'].value_counts() / df['city'].notna().sum() * 100, 2),
     'City Index':df.groupby(by='city').mean()['city_development_index']}
).head(15).sort_values('Total', ascending=False)

,Total,% of Total,City Index
city_103,4828,22.68,0.920
city_114,1491,7.00,0.926
city_102,334,1.57,0.804
city_104,328,1.54,0.924
city_100,296,1.39,0.887
city_11,277,1.30,0.550
city_10,97,0.46,0.895
city_105,90,0.42,0.794
city_101,85,0.40,0.558
city_115,61,0.29,0.789


There are 123 different cities total, so encoding might not be possible.

### 3) City development index

In [90]:
df.city_development_index.describe()

count    21287.000000
mean         0.828462
std          0.123537
min          0.448000
25%          0.739000
50%          0.903000
75%          0.920000
max          0.949000
Name: city_development_index, dtype: float64

### 4) Gender

In [91]:
len(df.gender), df.gender.notna().sum()

(21287, 16271)

In [92]:
# Gender distribution (of not null values)
pd.DataFrame(
    {'Total':df['gender'].value_counts(),
     '%':round(df['gender'].value_counts() / df['gender'].notna().sum() * 100, 2)}
)

,Total,%
Male,14681,90.23
Female,1375,8.45
Other,215,1.32


In [93]:
cat_cols.append('gender')

### 5) Relevent experience

In [94]:
df['relevent_experience'].value_counts()

Has relevent experience    15316
No relevent experience      5971
Name: relevent_experience, dtype: int64

In [95]:
# Change relevant experience to binary values
df['relevent_experience'] = np.where(df['relevent_experience']=='Has relevent experience', 1, 0)

### 6) Enrolled university

In [96]:
df['enrolled_university'].value_counts()

no_enrollment       15336
Full time course     4192
Part time course     1342
Name: enrolled_university, dtype: int64

In [97]:
# Rename values
df['enrolled_university'] = np.where(df['enrolled_university']=='Full time course', 'full_time', df['enrolled_university'])
df['enrolled_university'] = np.where(df['enrolled_university']=='Part time course', 'part_time', df['enrolled_university'])

df['enrolled_university'].value_counts()

no_enrollment    15336
full_time         4192
part_time         1342
Name: enrolled_university, dtype: int64

In [98]:
cat_cols.append('enrolled_university')

### 7) Education level

In [99]:
df['education_level'].value_counts()

Graduate          12867
Masters            4857
High School        2239
Phd                 468
Primary School      344
Name: education_level, dtype: int64

In [100]:
cat_cols.append('education_level')

### 8) Major discipline

In [101]:
# Rename column
df.rename(columns={'major_discipline':'major'}, inplace=True)

In [102]:
df['major'].value_counts()

STEM               16113
Humanities           749
Other                421
Business Degree      364
Arts                 270
No Major             245
Name: major, dtype: int64

In [103]:
cat_cols.append('major')

### 9) Experience

In [104]:
pd.DataFrame(df['experience'].value_counts())

,experience
>20,3669
5,1593
4,1548
3,1508
6,1346
2,1255
7,1144
9,1093
10,1081
8,884


Years of experience can be binned into smaller groups

In [105]:
def exp_range(exp):
    """Map to bin years of experience"""
    if exp in ['<1', '1']:
        return('<=1')
    elif exp in ['2', '3', '4', '5']:
        return('2-5')
    elif exp in ['6', '7', '8', '9', '10']:
        return('6-10')
    elif exp in ['11', '12', '13', '14', '15']:
        return('11-15')
    elif exp in ['16', '17', '18', '19', '20']:
        return('16-20')
    elif exp in ['>20']:
        return('>20')
    else:
        return(np.nan)

In [106]:
df['exp_range'] = df['experience'].apply(exp_range)
df[['experience', 'exp_range']].head(10)

,experience,exp_range
0,>20,>20
1,15,11-15
2,5,2-5
3,<1,<=1
4,>20,>20
5,11,11-15
6,5,2-5
7,13,11-15
8,7,6-10
9,17,16-20


In [107]:
df['exp_range'].value_counts()

2-5      5904
6-10     5548
>20      3669
11-15    3135
16-20    1760
<=1      1201
Name: exp_range, dtype: int64

In [108]:
# Add this column to list of columns to be ordinally encoded
ord_cols.append('exp_range')

In [109]:
# Drop the original experience column
df.drop(columns='experience', inplace=True)

### 10) Company size

In [110]:
df['company_size'].value_counts()

50-99        3421
100-500      2889
10000+       2236
10/49        1643
<10          1471
1000-4999    1471
500-999       965
5000-9999     631
Name: company_size, dtype: int64

In [111]:
# Fix value 10/49 so that it's 10-49
df['company_size'] = np.where(df['company_size']=='10/49', '10-49', df['company_size'])

# Change 100-500 to 100-499
df['company_size'] = np.where(df['company_size']=='100-500', '100-499', df['company_size'])

df[['company_size']][15:25]

,company_size
15,10-49
16,50-99
17,5000-9999
18,10000+
19,NaN
20,100-499
21,100-499
22,NaN
23,1000-4999
24,1000-4999


In [112]:
ord_cols.append('company_size')

### 11) Company type

In [113]:
df['company_type'].value_counts()

Pvt Ltd                10958
Funded Startup          1098
Public Sector           1082
Early Stage Startup      668
NGO                      574
Other                    133
Name: company_type, dtype: int64

In [114]:
#Rename some values

df['company_type'] = np.where(df['company_type']=='Pvt Ltd', 'private_limited', df['company_type'])
df['company_type'] = np.where(df['company_type']=='Funded Startup', 'startup_funded', df['company_type'])
df['company_type'] = np.where(df['company_type']=='Early Stage Startup', 'startup_earlystage', df['company_type'])
df['company_type'] = np.where(df['company_type']=='Public Sector', 'public_sector', df['company_type'])

In [115]:
cat_cols.append('company_type')

### 12) Last new job

In [116]:
df['last_new_job'].value_counts()

1        8924
>4       3643
2        3242
never    2710
3        1157
4        1148
Name: last_new_job, dtype: int64

In [117]:
ord_cols.append('last_new_job')

### 13) Training hours

In [118]:
df['training_hours'].value_counts()

28     354
18     332
12     332
22     307
50     307
      ... 
294      6
234      6
272      6
286      5
238      4
Name: training_hours, Length: 241, dtype: int64

### 14) Target

In [119]:
df['target'].value_counts()

0.0    14381
1.0     4777
Name: target, dtype: int64

### 15) Lists for encoding

In [120]:
# Affirm column names are in appropriate lists
cat_cols, ord_cols

(['gender', 'enrolled_university', 'education_level', 'major', 'company_type'],
 ['exp_range', 'company_size', 'last_new_job'])

# II. Save Clean df

In [121]:
df.shape

(21287, 14)

In [122]:
df.to_csv('../data/df_clean_nulls.csv')

In [123]:
# Create a secondary df where nulls are filled with 'missing' string for EDA

df_nonulls = df.drop('target', axis=1).fillna('MISSING')
df_nonulls['target'] = df['target']
df_nonulls.isna().sum()

enrollee_id                  0
city                         0
city_development_index       0
gender                       0
relevent_experience          0
enrolled_university          0
education_level              0
major                        0
company_size                 0
company_type                 0
last_new_job                 0
training_hours               0
exp_range                    0
target                    2129
dtype: int64

In [124]:
df_nonulls.shape

(21287, 14)

In [125]:
df_nonulls.to_csv('../data/df_clean_nonulls.csv')